<a href="https://colab.research.google.com/github/Janak-Khadka/WeatherWise-Janak-Khadka-23417861/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai


In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

## 🌤️ Weather Data Functions

In [4]:
# 🌤️ Weather Data Functions

class WeatherDataFetcher:
    def __init__(self, api_key="fba5b82a63a4969258fd9954ecf8a781"):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5"

    def get_weather_data(self, location, forecast_days=5):

        try:
            # Get current weather from OpenWeatherMap API
            current_data = self._get_current_weather(location)

            # Get forecast data from OpenWeatherMap API
            forecast_data = self._get_forecast(location, forecast_days)

            return {
                'location': location,
                'current': current_data,
                'forecast': forecast_data,
                'timestamp': datetime.now().isoformat()
            }

        except Exception as e:
            print(f"Error fetching weather data: {e}")
            # Return sample data if API fails
            return self._get_sample_data(location, forecast_days)

    def _get_current_weather(self, location):
        """Fetch current weather data from OpenWeatherMap API"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                'q': location,
                'appid': self.api_key,
                'units': 'metric'
            }

            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            return {
                'temperature': data['main']['temp'],
                'feels_like': data['main']['feels_like'],
                'humidity': data['main']['humidity'],
                'pressure': data['main']['pressure'],
                'wind_speed': data['wind']['speed'],
                'weather_main': data['weather'][0]['main'],
                'weather_description': data['weather'][0]['description'],
                'visibility': data.get('visibility', 'N/A'),
                'timestamp': datetime.now().isoformat()
            }

        except Exception as e:
            print(f"API Error: {e}")
            return self._get_demo_current_weather()

    def _get_forecast(self, location, days):
        """Fetch forecast data from OpenWeatherMap API"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                'q': location,
                'appid': self.api_key,
                'units': 'metric'
            }

            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()

            forecast = []
            for item in data['list'][:days * 8]:
                forecast.append({
                    'datetime': datetime.fromtimestamp(item['dt']).isoformat(),
                    'temperature': item['main']['temp'],
                    'feels_like': item['main']['feels_like'],
                    'humidity': item['main']['humidity'],
                    'weather_main': item['weather'][0]['main'],
                    'weather_description': item['weather'][0]['description'],
                    'wind_speed': item['wind']['speed'],
                    'pop': item.get('pop', 0)
                })

            return forecast

        except Exception as e:
            print(f"API Error: {e}")
            return self._get_demo_forecast(days)

    def _get_demo_current_weather(self):
        """Provide demo data when API is unavailable"""
        return {
            'temperature': 22.5,
            'feels_like': 24.1,
            'humidity': 65,
            'pressure': 1013,
            'wind_speed': 3.5,
            'weather_main': 'Clear',
            'weather_description': 'clear sky',
            'visibility': 10000,
            'timestamp': datetime.now().isoformat()
        }

    def _get_demo_forecast(self, days):
        """Provide demo forecast data"""
        forecast = []
        base_time = datetime.now()

        for i in range(days * 8):
            forecast_time = base_time + timedelta(hours=i*3)
            forecast.append({
                'datetime': forecast_time.isoformat(),
                'temperature': 20 + 5 * (i % 3),
                'feels_like': 21 + 5 * (i % 3),
                'humidity': 60 + 10 * (i % 2),
                'weather_main': ['Clear', 'Clouds', 'Rain'][i % 3],
                'weather_description': ['clear sky', 'few clouds', 'light rain'][i % 3],
                'wind_speed': 2 + (i % 4),
                'pop': min(0.9, i * 0.1)
            })

        return forecast

    def _get_sample_data(self, location, days):
        """Fallback sample data"""
        return {
            'location': location,
            'current': self._get_demo_current_weather(),
            'forecast': self._get_demo_forecast(days),
            'timestamp': datetime.now().isoformat()
        }

# Initialize the weather fetcher with your real API key
weather_fetcher = WeatherDataFetcher("fba5b82a63a4969258fd9954ecf8a781")

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.